In [1]:
%pip install -r ../requirements.txt

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----------------- ---------------------- 0.8/1.8 MB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 3.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import base64
import io
import logging
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import glob
import os
import webbrowser
import locale
from datetime import datetime, timedelta


In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
locale.setlocale(locale.LC_ALL, 'es_ES.UTF-8')

'es_ES.UTF-8'

In [5]:
def format_date(x, pos=None):
    date = mdates.num2date(x)
    if date.month == 1 and date.day == 1:
        return f"{date.year}"
    return ""

In [6]:
def format_euro(x, p):
    return locale.format_string('%.0f €', x, grouping=True)

In [7]:
def load_most_recent_csv(folder_path):
    # Use glob to find all CSV files in the specified folder
    csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
    
    # Get the most recent file based on the modification time
    if csv_files:
        latest_file = max(csv_files, key=os.path.getmtime)
        print(f"Loading the most recent file: {latest_file}")
        return pd.read_csv(latest_file)
    else:
        print("No CSV files found in the specified folder.")
        return None


In [8]:
folder_path = '../data'
data = load_most_recent_csv(folder_path)


Loading the most recent file: ../data\data_bank_trx.csv


In [9]:
print(data.head())


   profile   name surname  birth_date        dni                       email  \
0  buyer_1  Aitor   López  1996-08-10  06980792Q  aitor.lopez@smartpixel.com   
1  buyer_1  Aitor   López  1996-08-10  06980792Q  aitor.lopez@smartpixel.com   
2  buyer_1  Aitor   López  1996-08-10  06980792Q  aitor.lopez@smartpixel.com   
3  buyer_1  Aitor   López  1996-08-10  06980792Q  aitor.lopez@smartpixel.com   
4  buyer_1  Aitor   López  1996-08-10  06980792Q  aitor.lopez@smartpixel.com   

   password                    iban   assets             trx_date trx_city  \
0    628678  ES27209522028598818380  3314.42  2022-01-01T18:47:44   Bilbao   
1    628678  ES27209522028598818380  3314.42  2022-01-01T14:07:19   Bilbao   
2    628678  ES27209522028598818380  3314.42  2022-01-01T18:28:59   Bilbao   
3    628678  ES27209522028598818380  3314.42  2022-01-02T14:07:37   Bilbao   
4    628678  ES27209522028598818380  3314.42  2022-01-04T10:18:55   Bilbao   

   trx_type    trx_cat  trx_amount  balance  
0   

In [10]:
df = data.copy()


In [11]:
logger.info(f"DataFrame info:\n{df.info()}")
logger.info(f"DataFrame head:\n{df.head()}")
logger.info(f"Unique profiles: {df['profile'].unique()}")

INFO:__main__:DataFrame info:
None
INFO:__main__:DataFrame head:
   profile   name surname  birth_date        dni                       email  \
0  buyer_1  Aitor   López  1996-08-10  06980792Q  aitor.lopez@smartpixel.com   
1  buyer_1  Aitor   López  1996-08-10  06980792Q  aitor.lopez@smartpixel.com   
2  buyer_1  Aitor   López  1996-08-10  06980792Q  aitor.lopez@smartpixel.com   
3  buyer_1  Aitor   López  1996-08-10  06980792Q  aitor.lopez@smartpixel.com   
4  buyer_1  Aitor   López  1996-08-10  06980792Q  aitor.lopez@smartpixel.com   

   password                    iban   assets             trx_date trx_city  \
0    628678  ES27209522028598818380  3314.42  2022-01-01T18:47:44   Bilbao   
1    628678  ES27209522028598818380  3314.42  2022-01-01T14:07:19   Bilbao   
2    628678  ES27209522028598818380  3314.42  2022-01-01T18:28:59   Bilbao   
3    628678  ES27209522028598818380  3314.42  2022-01-02T14:07:37   Bilbao   
4    628678  ES27209522028598818380  3314.42  2022-01-04T10:18:5

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9088 entries, 0 to 9087
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   profile     9088 non-null   object 
 1   name        9088 non-null   object 
 2   surname     9088 non-null   object 
 3   birth_date  9088 non-null   object 
 4   dni         9088 non-null   object 
 5   email       9088 non-null   object 
 6   password    9088 non-null   int64  
 7   iban        9088 non-null   object 
 8   assets      9088 non-null   float64
 9   trx_date    9088 non-null   object 
 10  trx_city    9088 non-null   object 
 11  trx_type    9088 non-null   object 
 12  trx_cat     9088 non-null   object 
 13  trx_amount  9088 non-null   float64
 14  balance     9088 non-null   float64
dtypes: float64(3), int64(1), object(11)
memory usage: 1.0+ MB


In [12]:
# Check if 'trx_date' and 'balance' columns exist
if 'trx_date' not in df.columns or 'balance' not in df.columns:
    logger.error("Required columns 'trx_date' or 'balance' not found in the DataFrame")
    raise ValueError("Missing required columns in DataFrame")

# Ensure 'trx_date' is in datetime format
if not pd.api.types.is_datetime64_any_dtype(df['trx_date']):
    logger.info("Converting 'trx_date' to datetime")
    df['trx_date'] = pd.to_datetime(df['trx_date'], errors='coerce')

# Check for any NaT values in trx_date after conversion
if df['trx_date'].isna().any():
    logger.warning("NaT values found in 'trx_date' column after conversion")

# Set the style for the plots
plt.style.use('dark_background')

INFO:__main__:Converting 'trx_date' to datetime


In [13]:
image_base64_list = []
for profile in df['profile'].unique():
    try:
        profile_data = df[df['profile'] == profile].sort_values('trx_date')
        logger.info(f"Generating graph for profile: {profile}")
        logger.info(f"Profile data shape: {profile_data.shape}")
        logger.info(f"Profile data head:\n{profile_data.head()}")
        
        fig, ax = plt.subplots(figsize=(12, 6))
        sns.lineplot(x='trx_date', y='balance', data=profile_data, marker='o', ax=ax)
        
        ax.set_xlabel('Year', fontsize=12, color='white')
        ax.set_ylabel('Balance', fontsize=12, color='white')
        
        # Format x-axis
        start_date = profile_data['trx_date'].min()
        end_date = profile_data['trx_date'].max()
        
        # Adjust start_date to the beginning of its year
        start_date = start_date.replace(month=1, day=1)
        # Adjust end_date to the end of its year
        end_date = end_date.replace(month=12, day=31)
        
        # Generate dates for every 6 months
        dates = []
        current_date = start_date
        while current_date <= end_date:
            dates.append(current_date)
            # Add 6 months
            if current_date.month == 1:
                current_date = current_date.replace(month=7)
            else:
                current_date = current_date.replace(year=current_date.year + 1, month=1)
        
        # Extend the x-axis limits slightly to show the first year clearly
        ax.set_xlim(start_date - timedelta(days=15), end_date + timedelta(days=15))
        ax.xaxis.set_major_locator(mdates.YearLocator())
        ax.xaxis.set_major_formatter(ticker.FuncFormatter(format_date))
        ax.xaxis.set_minor_locator(mdates.MonthLocator([1, 7]))  # Ubicar líneas cada 6 meses
        
        # Add vertical lines for each 6 months
        for date in dates:
            ax.axvline(x=date, color='gray', linestyle='--', alpha=0.5)
        
        plt.xticks(color='white')
        
        # Format y-axis with European number format and € after the number
        ax.yaxis.set_major_formatter(ticker.FuncFormatter(format_euro))
        plt.yticks(color='white')
        
        # Add grid for y-axis only
        ax.grid(True, axis='y', linestyle='--', alpha=0.7, color='gray')
        
        # Find min and max points
        min_point = profile_data['balance'].min()
        max_point = profile_data['balance'].max()
        
        # Add horizontal lines for min, max, and balance 0
        ax.axhline(y=min_point, color='yellow', linestyle='--', alpha=0.7)
        ax.axhline(y=max_point, color='green', linestyle='--', alpha=0.7)
        ax.axhline(y=0, color='red', linewidth=2, alpha=0.7)  # Línea roja más gruesa en balance 0
        
        # Annotate min and max points without arrows
        ax.text(profile_data['trx_date'].iloc[-1], min_point, 
                f'Min: {format_euro(min_point, None)}',
                color='yellow', va='bottom', ha='right')
        
        ax.text(profile_data['trx_date'].iloc[-1], max_point, 
                f'Max: {format_euro(max_point, None)}',
                color='green', va='top', ha='right')
        
        # Ensure all spines are visible
        for spine in ax.spines.values():
            spine.set_edgecolor('white')
        
        plt.tight_layout()
        
        # Save plot to a bytes buffer
        buffer = io.BytesIO()
        plt.savefig(buffer, format='png', facecolor='#1E1E1E', edgecolor='none')
        buffer.seek(0)
        
        # Encode the bytes as base64
        image_base64 = base64.b64encode(buffer.getvalue()).decode()
        image_base64_list.append((profile, image_base64))
        
        plt.close(fig)
        logger.info(f"Successfully generated graph for profile: {profile}")
    except Exception as e:
        logger.error(f"Error generating graph for profile {profile}: {str(e)}", exc_info=True)

INFO:__main__:Generating graph for profile: buyer_1
INFO:__main__:Profile data shape: (1171, 15)
INFO:__main__:Profile data head:
   profile   name surname  birth_date        dni                       email  \
1  buyer_1  Aitor   López  1996-08-10  06980792Q  aitor.lopez@smartpixel.com   
2  buyer_1  Aitor   López  1996-08-10  06980792Q  aitor.lopez@smartpixel.com   
0  buyer_1  Aitor   López  1996-08-10  06980792Q  aitor.lopez@smartpixel.com   
3  buyer_1  Aitor   López  1996-08-10  06980792Q  aitor.lopez@smartpixel.com   
4  buyer_1  Aitor   López  1996-08-10  06980792Q  aitor.lopez@smartpixel.com   

   password                    iban   assets            trx_date trx_city  \
1    628678  ES27209522028598818380  3314.42 2022-01-01 14:07:19   Bilbao   
2    628678  ES27209522028598818380  3314.42 2022-01-01 18:28:59   Bilbao   
0    628678  ES27209522028598818380  3314.42 2022-01-01 18:47:44   Bilbao   
3    628678  ES27209522028598818380  3314.42 2022-01-02 14:07:37   Bilbao   
4   

In [14]:
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Profile Balance Report</title>
    <style>
        body { 
            font-family: Arial, sans-serif; 
            background-color: #1E1E1E; 
            color: white;
            text-align: center;
            padding: 20px;
        }
        .graph-container { 
            margin-bottom: 20px;
            display: inline-block;
        }
        img { 
            max-width: 100%; 
            height: auto; 
        }
        h1, h2 {
            color: #e0e0e0;
        }
    </style>
</head>
<body>
    <h1>Profile Balance Report</h1>
"""

for profile, image_base64 in image_base64_list:
    html_content += f"""
    <div class="graph-container">
        <h2>Profile: {profile}</h2>
        <img src="data:image/png;base64,{image_base64}" alt="Balance graph for {profile}">
    </div>
    """

html_content += """
</body>
</html>
"""

In [15]:
output_dir = '../data_report'
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, 'profile_balance_report.html')

try:
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(html_content)
    logger.info(f"HTML report saved successfully to {output_file}")
except Exception as e:
    logger.error(f"Error saving HTML report: {str(e)}")


INFO:__main__:HTML report saved successfully to ../data_report\profile_balance_report.html


In [16]:
try:
    webbrowser.open('file://' + os.path.realpath(output_file))
    logger.info("Opened HTML report in default web browser")
except Exception as e:
    logger.error(f"Error opening HTML report in browser: {str(e)}")

print("Please check the console for detailed logs and error messages.")

INFO:__main__:Opened HTML report in default web browser


Please check the console for detailed logs and error messages.
